In [2]:
import subprocess
import requests
from pystac_client import Client

In [3]:
catalog = Client.open("http://ec2-54-172-212-55.compute-1.amazonaws.com/api/v1/pgstac/")

# Washington, DC
bbox = [-76.964657, 38.978967, -76.928008, 39.002783] 
search = catalog.search(
    collections=["landsat-c2l1"],
    bbox=bbox,
)

items = search.get_all_items()
item = items[0]

In [4]:
item_cid = item.assets["red"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]
item_cid

'QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To'

In [5]:
def get_data(cid):
    CID_FOUND = None
    try:
        IPFS_PRESENT = subprocess.check_output(["ipfs", "version"]).decode("utf-8")
        print(f"Local IPFS Node Detected! {IPFS_PRESENT}")
        try:
            CID_FOUND = subprocess.check_output(["ipfs", "pin", "ls", cid])
            print(f"CID: {item_cid} found in local IPFS node")
            CID_FOUND = subprocess.check_output(["ipfs", "cat", cid])
            return CID_FOUND
        except subprocess.CalledProcessError as e:
            print(e.output.decode("utf-8"))

            try:
                print("Attempting to fetch CID from IPFS network using local IPFS node")
                CID_FOUND = subprocess.check_output(["ipfs", "cat", cid])
                print(f"CID: {item_cid} found on IPFS network via local IPFS node")
                print("Pinning CID to local IPFS node")
                print(subprocess.check_output(["ipfs", "pin", "add", cid]))
                return CID_FOUND
            except subprocess.CalledProcessError as e:
                print(f"CID: {cid} not found anywhere on IPFS. Aborting...")
                return
    except subprocess.CalledProcessError as e:
        print(e.output.decode("utf-8"))
        try:
            CID_FOUND = requests.get(f"https://ipfs.io/ipfs/{cid}").content
            print(f"CID: {item_cid} found on IPFS network via Gateway")
            subprocess.check_output(["ipfs", "add", "--pin", CID_FOUND])
            print("Pinned cid to local IPFS node")

        except requests.exceptions.RequestException as e:
            print(e.output.decode("utf-8"))
            return
        
    return CID_FOUND


data = get_data(item_cid)

Local IPFS Node Detected! ipfs version 0.18.1

CID: QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To found in local IPFS node


 82.62 MiB / 82.62 MiB  100.00% 0sK
